# **Project - SOAP Note Generation**

In [1]:
# install necessary libraries
#!pip install -r requirements.txt
!pip install openai
!pip install transformers
!pip install langchain
!pip install langchain-openai
!pip install langchain-community
!pip install datasets
!pip install chromadb
!pip install sentence-transformers
!pip install faiss-cpu
!pip install langchain-mistralai
!pip install langchain-google-genai
!pip install evaluate
!pip install rouge_score
!pip install datasets
!pip install streamlit
!pip install langchain-anthropic


[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


...


[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
#Load Libraries
import os
import openai
from langchain_openai import ChatOpenAI
from datasets import load_dataset
import csv
import pandas as pd
from langchain.docstore.document import Document
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma, FAISS
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage
from langchain_core.messages import AIMessage
import evaluate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_anthropic import ChatAnthropic
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings

c:\Users\deepa\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
os.environ["OPENAI_API_KEY"] = "Enter you OpenAI Key"
llm = ChatOpenAI(model="gpt-4")

In [5]:
#Load Dataset
dataset = load_dataset("adesouza1/soap_notes")

In [6]:
# Select train from the dataset, meta columns and columns which need be embeded
train=pd.DataFrame(dataset["train"])
metadata_col=['soap_notes']
embed_col=['patient_convo']

In [7]:
# Load Dataset in Langchain docs, such that they splitted by row soap notes as meta data and patient convo as embed
docs=[]
for index, row in train.iterrows():
  to_metadata = {col: row[col] for col in metadata_col if col in row}
  values_to_embed = {k: row[k] for k in embed_col}
  to_embed = "\n".join(f"{k.strip()}: {v.strip()}" for k, v in values_to_embed.items())
  newDoc = Document(page_content=to_embed, metadata=to_metadata)
  docs.append(newDoc)

In [8]:
#Loading a sample conversation for testing
conversation= "Physician: Good morning, Ivan. How are you feeling today? Patient: Hi, Dr. Cole. I've been feeling pretty miserable lately. I've had this constant ear pain, and it's been affecting my sleep and overall well-being. Physician: I see. Can you tell me more about the ear pain? Is it sharp, dull, or throbbing? And which ear is affected? Patient: It's a dull, aching pain in both ears, but the left ear is worse. It's been persistent for the past few weeks, and it's been getting worse. Physician: Okay. Have you experienced any other symptoms like fever, drainage, or difficulty hearing? Patient: Yes, I've had some hearing loss, and my ears feel plugged. I've also had some drainage, but it's not too much. And yes, I've had a low-grade fever on and off. Physician: I understand. Have you noticed any triggers that make the ear pain worse, such as allergies or sinus pressure? Patient: Yes, definitely. Whenever I have allergies, my ear pain seems to worsen. And I've been having a lot of sinus pressure and congestion lately. Physician: That's helpful to know. Have you tried any treatments or medications for your symptoms? Patient: I've been using over-the-counter pain relievers and decongestants, but they don't seem to be helping much. Physician: I see. Based on what you've shared, it sounds like you might have developed a couple of complications related to your allergies and sinus issues. We'll need to address these issues and prevent them from becoming chronic problems. Patient: That sounds good. What do you recommend? Physician: First, we'll need to update your vaccines to ensure you're protected against influenza and other diseases. We'll also schedule you for a colonoscopy to rule out any potential issues. In addition, we'll perform a hearing test to assess any hearing loss and determine the best course of action. Patient: Okay, that makes sense. Physician: We'll also consider prescribing medications to manage your allergies and sinus pressure. And I'll consult with Dr. Smith, your ENT specialist, to get his input on your ear issues. Patient: That sounds like a good plan. Physician: Great. We'll coordinate with Dr. Smith's office to schedule an appointment for you. In the meantime, I recommend that you avoid any known allergens and take steps to minimize your exposure to dust, pollen, and other irritants. Patient: Okay, I'll do that. Thank you, Dr. Cole. Physician: You're welcome, Ivan. We'll follow up with you soon to discuss the results of your tests and develop a comprehensive treatment plan."
# actual soap notes for the above sample conversation
act_soap="Subjective: Ivan Bailey reports feeling miserable due to constant ear pain that has been affecting his sleep and overall well-being. He describes the pain as dull and aching, worse in the left ear, with associated hearing loss, ear plugging, drainage, and low-grade fever. He notes that allergies exacerbate the symptoms, along with sinus pressure and congestion. Over-the-counter pain relievers and decongestants have provided minimal relief. Objective: Ivan Bailey presents with symptoms of chronic ear pain, hearing loss, drainage, and congestion. Physical examination reveals signs of bilateral ear inflammation. Vital signs are within normal limits. Immunization status is up to date, with pending results for a colonoscopy. ENT specialist Dr. Robert Smith recommends a hearing test to monitor for potential hearing loss. Assessment: Ivan Bailey is experiencing recurrent ear infections with complications related to allergies and sinus issues. Further evaluation is needed to determine the extent of the infection and associated hearing loss. Collaborative care with the ENT specialist is recommended to address the ear issues comprehensively. Plan: 1. Update vaccines for influenza and tetanus. 2. Schedule a colonoscopy to rule out potential issues. 3. Perform a hearing test to assess hearing loss. 4. Prescribe medications to manage allergies and sinus pressure. 5. Consult with ENT specialist Dr. Robert Smith for further evaluation and treatment recommendations. 6. Advise Ivan to avoid allergens and irritants to minimize symptoms. 7. Follow up with Ivan to discuss test results and develop a comprehensive treatment plan."

In [9]:
# load bleu and rouge metrics for evaluation

bleu= evaluate.load("bleu")
rouge = evaluate.load('rouge')

## **RAGS**

In [10]:
# create embedding the splitted docs using open source model
embeddings = SentenceTransformerEmbeddings(model_name="BAAI/bge-small-en-v1.5", model_kwargs = {'device': 'cpu'})
#using FAISS as vectorstore/vector_index
vectorstore = FAISS.from_documents(docs, embeddings)

#save vectorestore, which can be utilized in the chatbot app.
vectorstore.save_local("vectorstore")

c:\Users\deepa\AppData\Local\Programs\Python\Python312\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(


In [ ]:
## Deprioritized as this one exhaust the free quota of the LLM

#from langchain.retrievers.multi_query import MultiQueryRetriever
#retriever = MultiQueryRetriever.from_llm(
#    retriever=vectorstore.as_retriever(), llm=llm
#)

#unique_docs = retriever.get_relevant_documents(conversation)
#len(unique_docs)

In [14]:
# using vector index based

docs = vectorstore.similarity_search(conversation)

In [15]:
# sample conv_soap_pair for context in Rags
conv_soap_pair=f"""Conversation: \n {docs[0].page_content} \n SOAP Notes: \n {docs[0].metadata["soap_notes"]}"""

In [11]:
# load Prompt Template for Generation using RAGs

prompt_rag = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are expert in writing SOAP notes from the conversation.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain_rag = prompt_rag | llm

In [17]:
# call rags on sample conversation and sample conv_soap_pair
response_rag = chain_rag.invoke(
    {"messages": [HumanMessage(content=f"Write SOAP Notes from the provided conversation: \n {conversation}.Here is an example of a conversation and its respective SOAP notes {conv_soap_pair}. Use the same format of the soap notes as provided in the example")], 
     }
    )


print(response_rag.content)

Subjective:
Ivan has been experiencing a dull ache in both ears for the past few weeks, with the left ear being more severe. He has also had some hearing loss, feeling of plugged ears, and drainage. He has had a low-grade fever on and off. He mentions a correlation between his allergies and the worsening of his ear pain. He also indicates sinus pressure and congestion. Over-the-counter pain relievers and decongestants have not been very effective for him.

Objective:
Ivan appears to be in moderate distress due to his symptoms. His physical examination shows inflammation in the ears, more pronounced in the left ear. There is mild hearing loss and some effusion noted. Vital signs show a low-grade fever.

Assessment:
Ivan's symptoms suggest a possible chronic ear infection, possibly a complication of his allergies and sinus issues. The persistent nature of his symptoms, despite over-the-counter treatments, implies that further medical intervention is required.

Plan:
1. Schedule an appoin

In [ ]:
# Compute the BLEU score
results = bleu.compute(predictions=[response_rag.content], references=[act_soap])

# Print the results
print(results)

{'bleu': 0.2863342985662251, 'precisions': [0.6496598639455783, 0.3515358361774744, 0.2089041095890411, 0.140893470790378], 'brevity_penalty': 1.0, 'length_ratio': 1.03886925795053, 'translation_length': 294, 'reference_length': 283}


In [18]:
results = rouge.compute(predictions=[response_rag.content], references=[act_soap])

# Print the results
print(results)

{'rouge1': 0.6015625, 'rouge2': 0.2823529411764706, 'rougeL': 0.40625, 'rougeLsum': 0.41015625}


## **Direct Prompting - One Shot**

In [15]:
# load conversation example and soap notes example for one shot
conversation_x=docs[-1].page_content
soap_x=docs[-1].metadata["soap_notes"]

In [16]:
def one_shot_prompt(conversation,conversation_ex=conversation_x,soap_ex=soap_x):
  prompt=f"""Write SOAP Notes from the provided conversation.
  <conversation>
  Here is the conversation from which you need to write SOAP notes:
  {conversation}
  </conversation>
  <example>
  Here is an example of conversation and its respective SOAP notes:
  Conversation: {conversation_x}
  SOAP Notes: {soap_x}
  </example>
  <format>
  Here is the format of the response
  Subjective (S): Patient's reported symptoms and medical history.
  Objective (O): Measurable and observable clinical data.
  Assessment (A): Professional interpretation and diagnosis.
  Plan (P): Strategy for treatment and management.
  </format>
"""
  return prompt

In [17]:
# load Prompt Template for Generation using direct prompting

prompt_direct = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are expert in writing SOAP notes from the conversation."
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain_direct = prompt_direct | llm

In [22]:
# Call LLM on one shot prompt using sample conversation, conversation_x and soap_note_x as example

response = chain_direct.invoke(
    {"messages": [HumanMessage(content=one_shot_prompt(conversation))]
     }
    )


print(response.content)

Subjective (S): 
The patient, Ivan, reported persistent dull aching pain in both ears, worse in the left, for a few weeks. He has also noticed some hearing loss, feelings of plugged ears, drainage, and intermittent low-grade fever. The patient also stated that his ear pain seems to worsen during allergy flare-ups and during periods of sinus pressure and congestion. He has been using over-the-counter pain relievers and decongestants without significant improvement.

Objective (O): 
At this point, no physical examination data or measurable observations have been provided in the conversation.

Assessment (A): 
The patient's symptoms suggest he may have developed complications related to his allergies and sinus issues, potentially including a middle ear infection or eustachian tube dysfunction, which may explain the ear pain, hearing loss, and drainage. 

Plan (P): 
1. Update the patient's vaccines to protect against influenza and other diseases.
2. Schedule a colonoscopy to rule out any p

In [23]:
# Compute the BLEU score
results = bleu.compute(predictions=[response.content], references=[act_soap])
# Print the results
print(results)

{'bleu': 0.2808310126678959, 'precisions': [0.6131386861313869, 0.3553113553113553, 0.22058823529411764, 0.14760147601476015], 'brevity_penalty': 0.96768687978665, 'length_ratio': 0.9681978798586572, 'translation_length': 274, 'reference_length': 283}


In [24]:
# Compute the BLEU score
results = rouge.compute(predictions=[response.content], references=[act_soap])
# Print the results
print(results)

{'rouge1': 0.5857740585774059, 'rouge2': 0.3151260504201681, 'rougeL': 0.4309623430962343, 'rougeLsum': 0.4476987447698745}


## **Evaluate on Test Data**

In [18]:
# load test data, loading on 10 to preserve free quota of llm

test=pd.DataFrame(dataset["test"]).loc[:10]

In [19]:
df_test_evaluate=pd.DataFrame(columns=['conversation','soap_notes', 'rag_retried', 'rag_response', 'direct_prompt_response', 'rag_rouge', 'direct_rouge','rag_bleu','direct_blue'])

In [20]:
for index, row in test.iterrows(): #iterate on all loaded rows
  conversation=row['patient_convo']
  soap_notes=row['soap_notes']
  unique_docs = vectorstore.similarity_search(conversation) # retrieve similar docs
  conv_soap_pair=f"""Conversation: \n {unique_docs[0].page_content} \n SOAP Notes: \n {unique_docs[0].metadata["soap_notes"]} """

  # call rag based generation
  response_rag = chain_rag.invoke(
    {"messages": [HumanMessage(content=f"Write SOAP Notes from the provided conversation: \n {conversation}.Here is an example of a conversation and its respective SOAP notes {conv_soap_pair}. Use the same format of the soap notes as provided in the example")], "context": conv_soap_pair
     }
    )

  #call direct promting generation (one shot)
  response_direct=response = chain_direct.invoke(
    {"messages": [HumanMessage(content=one_shot_prompt(conversation))]
     }
    )

  #calculate metrics
  rouge_rag = rouge.compute(predictions=[response_rag.content], references=[soap_notes])
  rouge_direct = rouge.compute(predictions=[response_direct.content], references=[soap_notes])
  bleu_rag = bleu.compute(predictions=[response_rag.content], references=[soap_notes])
  bleu_direct = bleu.compute(predictions=[response_direct.content], references=[soap_notes])
  print(index)
  df_test_evaluate.loc[index]=[conversation,soap_notes,conv_soap_pair,response_rag.content,response_direct.content,rouge_rag,rouge_direct,bleu_rag,bleu_direct]

0
1
2
3
4
5
6
7
8
9


KeyboardInterrupt: 

In [22]:
# save evaluated file
df_test_evaluate.to_csv('test_evaluate.csv')

In [27]:
print(test.patient_convo.values[0])


Physician: Good morning, Ivan. How are you feeling today?

Patient: Hi, Dr. Cole. I've been feeling pretty miserable lately. I've had this constant ear pain, and it's been affecting my sleep and overall well-being.

Physician: I see. Can you tell me more about the ear pain? Is it sharp, dull, or throbbing? And which ear is affected?

Patient: It's a dull, aching pain in both ears, but the left ear is worse. It's been persistent for the past few weeks, and it's been getting worse.

Physician: Okay. Have you experienced any other symptoms like fever, drainage, or difficulty hearing?

Patient: Yes, I've had some hearing loss, and my ears feel plugged. I've also had some drainage, but it's not too much. And yes, I've had a low-grade fever on and off.

Physician: I understand. Have you noticed any triggers that make the ear pain worse, such as allergies or sinus pressure?

Patient: Yes, definitely. Whenever I have allergies, my ear pain seems to worsen. And I've been having a lot of sinus 